In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import Select


In [4]:
url='https://weteach.edu.tw/my/'

options = Options()
options.headless = False
driverPath = r'/media/pancala/3T/code/moodleCrawl/geckodriver'
#driverPath = r'C:\Users\user\Documents\GitHub\moodleCrawl\geckodriver.exe'
driver = webdriver.Firefox(options=options, executable_path=driverPath)
#driver = webdriver.Firefox(executable_path=driverPath)

#headless模式
#options = webdriver.ChromeOptions()
#options.add_argument('--headless')
#driver = webdriver.Chrome(options=options, executable_path='chromedriver.exe')
#driver = webdriver.Chrome(executable_path='chromedriver.exe')

driver.set_page_load_timeout(60)
driver.get(url)


In [ ]:
#登入moodle
username = 'username'
password = 'password'
element = driver.find_element_by_id('login_username')
element.send_keys(username)
element = driver.find_element_by_id('login_password')
element.send_keys(password)

driver.find_element_by_css_selector(".btn-primary").click()


In [ ]:
#進入課程網址

In [9]:
#取得測驗的連結
id = 1735
#https://weteach.edu.tw/course/view.php?id=1735
courseurl = 'http://weteach.edu.tw/course/view.php?'
quizname=[]
quizurl=[]
for i in range(9):
    url = courseurl + 'id='+str(id) +'&section='+str(i+1)
    driver.get(url)
    #取得各section原始碼
    htmltext = driver.page_source

    soup=BeautifulSoup(htmltext,"html.parser")
    
    #搜尋所有含quiz的連結
    links = soup.find_all('a',{'href':re.compile('quiz')})
    for link in links:

        #取得href屬性前先檢查是否存在
        if 'href' in link.attrs:
            href = link['href']
            print(link.text)
            print(href)
            quizname.append(link.text)
            quizurl.append(href)
        else:
            href = None

細胞分裂 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43687
同源染色體 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43688
減數分裂 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43689
減數分裂-排序填圖題 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43690
減數分裂-子細胞染色體判斷 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43691
無性生殖 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43693
動物的有性生殖 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43699
人類的有性生殖 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43700
蛋的觀察 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43701
植物的有性生殖 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43702
遺傳的計算題 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43727
孟德爾的遺傳法則 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43728
性染色體 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43739
決定性別的機率 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43740
血型的遺傳 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43741
突變 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43746
人類的遺傳疾病 測驗卷
https://wet

In [10]:
#從url找id
quizid=[]
for quiz in quizurl:
    k = re.search(r'id=(.*)$', quiz)
    quizid.append(k.group(1))
quizid

['43687',
 '43688',
 '43689',
 '43690',
 '43691',
 '43693',
 '43699',
 '43700',
 '43701',
 '43702',
 '43727',
 '43728',
 '43739',
 '43740',
 '43741',
 '43746',
 '43747',
 '43751',
 '43752']

In [11]:
#進入試卷編修網址
                        #https://weteach.edu.tw/course/modedit.php?update=43489&return=0&sr=0
quizurlBase = 'https://weteach.edu.tw/course/modedit.php?update='
n = 0



gradecat = '2072'
#2074:A1 2065:A2 2067:A3 2072:B1 2070:B2  2069:B3
gradepass = 100  #及格分數
attempts = 2    #可測驗次數
grademethod = '1' 
#1:最高分數 2:平均分數 3:第一次作答  4:最後一次作答
for i in quizid:
    quizurl = quizurlBase+str(i)+'&return=1'
    
    driver.get(quizurl)

    #展開全部
    driver.find_element_by_css_selector('.collapseexpand').click()

    #修改及格分數
    element =  driver.find_element_by_id('id_gradepass')
    element.clear()
    element.send_keys(gradepass)

    
    s1 = Select(driver.find_element_by_id('id_gradecat'))
    s1.select_by_value(gradecat)
   
    
    #修改允許作答次數
    #搜尋作答次數的選單並點擊
    element = driver.find_element_by_id('id_attempts')
    element.click()
    element.send_keys(attempts)
    
    
    #修改評分方式
    s1 = Select(driver.find_element_by_id('id_grademethod'))
    s1.select_by_value(grademethod)
    
    
    #解開完成進度的選項
    #還沒有人作答時，此項須註解。等到有人作答後，才可以找到這個element
    #driver.find_element_by_id('id_unlockcompletion').click()
      
    #檢查是否勾選「需要及格分數」
    element = driver.find_element_by_id('id_completionpass')
    if element.get_attribute('checked') :
        #如果有勾選，就按一下，讓它取消勾選
        element.click()
        print('checked-->unchecked')
    else:
        print('unchecked')
    
    

    #儲存並顯示
    driver.find_element_by_id('id_submitbutton2').click()
    print('done:' + quizname[n])
    print(quizurl)
    n = n+1
    print(str(n)+"/"+str(len(quizid)))
    print('========')


checked-->unchecked
done:細胞分裂 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43687&return=1
1/19
checked-->unchecked
done:同源染色體 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43688&return=1
2/19
checked-->unchecked
done:減數分裂 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43689&return=1
3/19
checked-->unchecked
done:減數分裂-排序填圖題 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43690&return=1
4/19
checked-->unchecked
done:減數分裂-子細胞染色體判斷 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43691&return=1
5/19
checked-->unchecked
done:無性生殖 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43693&return=1
6/19
checked-->unchecked
done:動物的有性生殖 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43699&return=1
7/19
checked-->unchecked
done:人類的有性生殖 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43700&return=1
8/19
checked-->unchecked
done:蛋的觀察 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43701&return=1
9/19
checked-->unchecked
done:植物的有性生殖 測驗卷
https://weteac

## 以下是測試區

In [28]:
#處理單一連結的測試區

#進入試卷編修網址
url = 'https://weteach.edu.tw/course/modedit.php?update=43593'
driver.get(url)

#element = WebDriverWait(driver,3).until(expected_conditions.presence_of_element_located((By.ID,'id_gradepass')))
#element = driver.find_element_by_name('gradepass')

#展開全部
driver.find_element_by_css_selector('.collapseexpand').click()

#修改及格分數
element =  driver.find_element_by_id('id_gradepass')
element.clear()
element.send_keys('100')

#修改允許作答次數
#搜尋作答次數的選單並點擊
element = driver.find_element_by_id('id_attempts')
element.click()
element.send_keys('7')

#for option in driver.find_elements_by_tag_name('option'):
    #if option.text == '3':
    #    option.click()

grademethod = '2'
#修改評分方式
s1 = Select(driver.find_element_by_id('id_grademethod'))
s1.select_by_value(grademethod)

#element = driver.find_element_by_id('id_grademethod')
#element.click()
#element.send_keys( grademethod )

#解開完成進度的選項
driver.find_element_by_id('id_unlockcompletion').click()


#儲存並顯示
driver.find_element_by_id('id_submitbutton2').click()
print('done')
#element = driver.find_element_by_xpath('//*[@id="id_gradepass"]')


#finally:
    #driver.quit()


done


In [9]:
#進入試卷編修網址
url = 'https://weteach.edu.tw/course/modedit.php?update=43593'
driver.get(url)

#element = WebDriverWait(driver,3).until(expected_conditions.presence_of_element_located((By.ID,'id_gradepass')))
#element = driver.find_element_by_name('gradepass')

#展開全部
driver.find_element_by_css_selector('.collapseexpand').click()

#解開完成進度的選項
driver.find_element_by_id('id_unlockcompletion').click()


#檢查是否勾選「需要及格分數」
element = driver.find_element_by_id('id_completionpass')
if element.get_attribute('checked') :
    #如果有勾選，就按一下，讓它取消勾選
    element.click()
    print('checked-->unchecked')
else:
    print('unchecked')

checked
